# Spindle Freq from Central Gyrus

Here I tested the hypothesis that the spindle frequency jumps quickly around the central sulcus, from the precentral to the postcentral gyrus.

For each patient, I defined a line connecting the dots in the precentral gyrus and one in the postcentral gyrus. 
This should give me an idea of where the central sulcus is.
It turns out that the two lines are rather parallel, which is really convenient.

For each electrode (with spindles) I checked that it belonged to one of the brain regions of interest (I excluded the temporal cortex and occipital because in those cases the distance to the central sulcus is not meaningful).
However, defining the regions of interest was hard in the parietal cortex, because the supramarginal gyrus includes parts of the temporal cortex.

After that, I used the distance from both gyri to compute the position.
If the position is positive, it means that the distance to the postcentral gyrus is larger than the distance to the precentral gyrus (which means that the electrode is very frontal).

However the current implementation does not compute the correct distance because it's only the difference between the two lines, which is not very meaningful.

In [1]:
from phypno.attr import Freesurfer, Channels

from spgr.constants import (IMAGES_PATH,
                            REC_PATH,
                            FS_FOLDER,
                            HEMI_SUBJ, 
                            CHAN_TYPE, 
                            DATA_OPTIONS,
                            )
from spgr.read_data import get_chan_used_in_analysis

In [2]:
from spgr.constants import (HEMI_SUBJ,
                            CHAN_TYPE,
                            DATA_OPTIONS,
                            SPINDLE_OPTIONS,
                            )
from spgr.read_data import get_data
from spgr.detect_spindles import get_spindles
from spgr.spindle_source import get_morph_linear, get_chan_with_regions
from spgr.plot_spindles import plot_surf
from spgr.lmer_stats import lmer, lg

In [3]:
from phypno.viz import Viz3
SKIN_COLOR = (239, 208, 207, 240)

In [4]:
cool_regions = ('caudalanteriorcingulate',
 'caudalmiddlefrontal',
 'corpuscallosum',
 'frontalpole',
 'insula',
 'isthmuscingulate',
 'lateralorbitofrontal',
 'medialorbitofrontal',
 'paracentral',
 'parsopercularis',
 'parsorbitalis',
 'parstriangularis',
 'postcentral',
 'posteriorcingulate',
 'precentral',
 'precuneus',
 'rostralanteriorcingulate',
 'rostralmiddlefrontal',
 'superiorfrontal',
 'superiorparietal',
 )

# 'cuneus'
# 'supramarginal'
# 'inferiorparietal'

In [5]:
from numpy import *
from numpy import min, max
from visvis import plot, figure

In [6]:
def get_interp_line(data):
    datamean = data.mean(axis=0)
    uu, dd, vv = linalg.svd(data - datamean)
    linepts = vv[0] * mgrid[-50:50:2j][:, newaxis]
    return linepts + datamean, vv[0]

In [7]:
def line_from_label(surf, label, labelname):
    idx = label[2].index(labelname)
    x = label[0]
    data = surf.vert[where(x == idx)[0], ]
    linepts, weight = get_interp_line(data)
    return data, linepts

In [8]:
norm = linalg.norm

def distance_from_line(linepts, p):
    l2 = linepts[0]
    l1 = linepts[1]
    return norm(cross(l1 - l2, l2 - p)) / norm(l1 - l2)

In [9]:
def val_and_dist(fs, hemi, chan, spindles):

    surf = getattr(fs.read_brain(), hemi)
    label = fs.read_label(hemi)
    
    data, linepts_pre = line_from_label(surf, label, 'precentral')
    data, linepts_post = line_from_label(surf, label, 'postcentral')

    val = []
    dist_pre = []
    dist_post = []

    for good_chan in chan.chan:
        region = fs.find_brain_region(good_chan.xyz)[0]
        if [True for x in cool_regions if x in region]:
            val.append(spindles(trial=0, chan=good_chan.label))
            dist_pre.append(distance_from_line(linepts_pre, good_chan.xyz))
            dist_post.append(distance_from_line(linepts_post, good_chan.xyz))

    dist = array(dist_post) - array(dist_pre)
    val = array(val)

    return val, dist

In [10]:
all_val = {}

for subj, hemi in HEMI_SUBJ.items():

    chan = get_chan_used_in_analysis(subj, 'sleep', CHAN_TYPE, **DATA_OPTIONS)
    fs = Freesurfer(str(REC_PATH.joinpath(subj).joinpath(FS_FOLDER)))

    ref = 'avg'
    spindles = get_spindles(subj, chan_type=CHAN_TYPE, reref=ref, **SPINDLE_OPTIONS)
    spindles = spindles.to_data('peak_freq')

    val, dist = val_and_dist(fs, hemi, chan, spindles)
    
    figure()
    plot(dist, val, ls='', ms='.')
    all_val[subj] = dist, val

/home/gio/tools/venv/lib/python3.4/site-packages/visvis/text/text_freetype.py:698: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  data = np.array(data,dtype=np.ubyte).reshape(h,w,self.depth)
/home/gio/tools/venv/lib/python3.4/site-packages/visvis/text/text_freetype.py:201: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.data[y:y+height,x:x+width, :] = data


In [11]:
x = []
y = []
for x1, y1 in all_val.values():
    x.append(x1)
    y.append(y1)

x = hstack(x)
y = hstack(y)

In [12]:
def find_bin(bins, x1):
    idx = where(x1 >= bins)[0][-1] - 1
    return idx

In [13]:
bins = arange(-24, 24, 2)
val = empty(shape=(23,), dtype='object')
for i in range(23):
    val[i] = []
    
for x1, y1 in zip(x, y):
    idx = find_bin(bins, x1)
    val[idx].append(y1)

In [15]:
val_mean = array([mean(i) for i in val])
val_sd = array([std(i) / sqrt(len(i)) for i in val])

/home/gio/tools/venv/lib/python3.4/site-packages/visvis/text/text_freetype.py:698: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  data = np.array(data,dtype=np.ubyte).reshape(h,w,self.depth)
/home/gio/tools/venv/lib/python3.4/site-packages/visvis/text/text_freetype.py:201: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.data[y:y+height,x:x+width, :] = data


In [16]:
plot(bins[:-1] + 1, val_mean - val_sd)
plot(bins[:-1] + 1, val_mean + val_sd)

In [17]:
spindles = get_spindles(subj, chan_type=CHAN_TYPE, reref=ref, **SPINDLE_OPTIONS)


/home/gio/tools/venv/lib/python3.4/site-packages/visvis/text/text_freetype.py:698: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  data = np.array(data,dtype=np.ubyte).reshape(h,w,self.depth)
/home/gio/tools/venv/lib/python3.4/site-packages/visvis/text/text_freetype.py:201: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.data[y:y+height,x:x+width, :] = data


In [21]:
a = spindles.to_data('peak_freq')
s = spindles.to_data('peak_freq', std)

In [24]:
plot(a(trial=0), s(trial=0), ls='', ms='.')

/home/gio/tools/venv/lib/python3.4/site-packages/visvis/text/text_freetype.py:698: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  data = np.array(data,dtype=np.ubyte).reshape(h,w,self.depth)
/home/gio/tools/venv/lib/python3.4/site-packages/visvis/text/text_freetype.py:201: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.data[y:y+height,x:x+width, :] = data
